# First version to download Modern Slavery Statement PDFs.

Can handle HTML format slavery statements

In [2]:
%load_ext autoreload
%autoreload 2

In [31]:
import pandas as pd
import numpy as np
from os import getcwd, path
from requests.exceptions import HTTPError
import pickle
from time import sleep
import functools
import requests
from func_timeout import func_timeout, FunctionTimedOut
from bs4 import BeautifulSoup as soup

In [86]:
PROJECT_NAME = 'modern_slavery_registry'
PROJECT_PATH = f"{getcwd()[:getcwd().find(PROJECT_NAME)]}{PROJECT_NAME}"
DATA_PATH = f"{PROJECT_PATH}\\data"
SHEET_PATH = f"{DATA_PATH}\\sheets"
PICKLE_PATH = f"{DATA_PATH}\\pickles"
PDF_PATH = f"{DATA_PATH}\\pdfs"
TEXT_PATH = f"{DATA_PATH}\\texts"
MAX_DOWNLOAD_TIME = 5

In [5]:
def save_pickle(obj, file_name):
    pickle.dump(obj, file=open(f"{file_name}.pickle","wb"))
    
def load_pickle(file_name):
    return pickle.load(file=open(f"{file_name}.pickle","rb"))

In [89]:
pickle_name = "leftover_pdf_to_downloads_SIDs"
leftover_SIDs = load_pickle(file_name=f"{PICKLE_PATH}\\{pickle_name}")
TOTAL_SIDs = len(leftover_SIDs)
print(f"Total SIDs - PDFs left to download : {len(leftover_SIDs)}")

Total SIDs - PDFs left to download : 6043


In [90]:
df = pd.read_csv(f"{DATA_PATH}\modernslaveryregistry-2020-09-14.csv")
df.head()

,Company ID,Company,Is Publisher,Statement ID,URL,Override URL,Companies House Number,Industry,HQ,Is Also Covered,UK Modern Slavery Act,California Transparency in Supply Chains Act,Australia Modern Slavery Act,Period Covered
0,7676,"""K"" Line Holding Europe Limited",True,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05005018,Marine,United Kingdom,False,True,False,False,2018-2019
1,28660,"""K"" Line Bulk Shipping (UK) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,04830352,Marine,United Kingdom,True,True,False,False,2018-2019
2,28659,"""K"" Line (Europe) Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,05639474,Marine,United Kingdom,True,True,False,False,2018-2019
3,28661,"""K"" Line LNG Shipping Limited",False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,NaN,Marine,United Kingdom,True,True,False,False,2018-2019
4,28658,Polar LNG Shipping (UK) Limited,False,35092,https://img1.wsimg.com/blobby/go/7695baff-3f0f...,NaN,02205323,Marine,United Kingdom,True,True,False,False,2018-2019


In [91]:
TOTAL_UNIQUE_SIDs = len(np.unique(df['Statement ID'].values))
print(f"Total enteries : {len(df)}, "
      f"total unique statements : {TOTAL_UNIQUE_SIDs}")

Total enteries : 27531, total unique statements : 17799


In [92]:
def download_pdf_from_url(url, sid):
    """Download PDF for input url."""
    response = urllib.request.urlopen(url=url)
    file_name = f"SID-{statement_id}"
    file = open(f"{PDF_PATH}\\{file_name}.pdf", 'wb')
    file.write(response.read())
    file.close()

In [93]:
def get_pdf_url_from_sid(sid):
    """Return PDF url for input sid."""
    return df[df['Statement ID']==sid]['URL'].values[0]

In [179]:
def print_loop_no(total_sids, sid, itr, type_, type_itr, msg=""):
    """Print loop statements."""
    primary_text = f"Running - {itr}/{total_sids} , SID - {sid}"
    if type_ == "Success":
        primary_text = f"{primary_text} - Success - {type_itr}"
    elif type_ == "Failed":
        primary_text = f"{primary_text} - Failed - {type_itr}"
    else:
        primary_text = f"{primary_text} - Processed - {type_itr}"
    print(f"{'-'*8} {primary_text} - {msg} - {'-'*8}")        

In [95]:
def clean_text(text):
    """Clean and return input text."""
    text = text.replace("\n", " ")
    text = text.replace("\t", " ")
    return " ".join(text.split())

In [171]:
def is_response_html(response):
    if "<html>" in str(response) or "!DOCTYPE html" in str(response):
        return True
    return False

In [164]:
def is_response_pdf(response):
    if "%PDF" in str(response):
        return True
    return False

In [97]:
def clean_html_response(html_response):
    text = soup(str(html_response)).text
    text = clean_text(text)
    special_char = ["\\", "b'"]
    for char_ in special_char:
        text = text.replace(char_, " ")
    cleaned_text = []
    for word in text.split():
        if len(word) > 1:
            cleaned_text.append(word)
    cleaned_text = " ".join(cleaned_text) 
    return cleaned_text

In [98]:
def save_text(text, sid):
    """Save text in text file."""
    file_name = f"SID-{sid}" 
    file_obj = open(f"{TEXT_PATH}//{file_name}.txt", 'w', encoding='utf-8')
    file_obj.write(text)
    file_obj.close()

In [145]:
def save_pdf(pdf_response, sid):
    """Save text in text file."""
    file_name = f"SID-{sid}" 
    try:
        file_obj = open(f"{PDF_PATH}//{file_name}.pdf", 'w', encoding='utf-8')
        file_obj.write(pdf_response)
    except TypeError as type_error:
        try:
            file_obj = open(f"{PDF_PATH}//{file_name}.pdf", 'wb')
            file_obj.write(pdf_response)
        except Exception as e:
            print(e)
    file_obj.close()

In [185]:
saved_SIDs2 = []
failed_SIDs2 = []

In [ ]:
for i, sid in enumerate(leftover_SIDs):
    url = get_pdf_url_from_sid(sid)
    file_name = f"SID-{sid}"
    if sid not in saved_SIDs2 + failed_SIDs2:
        try:
            response = requests.get(url, timeout=10, allow_redirects=True).content
            if is_response_html(response=response): # if html response
                # save in text format if html response
                text_response = clean_html_response(html_response=response)
                text_response = clean_html_response(html_response=text_response)
                save_text(text=text_response, sid=sid)
                saved_SIDs2.append(sid)
                print_loop_no(total_sids=TOTAL_SIDs, 
                                    sid=sid,itr=i+1,
                                    type_="Success",
                                    type_itr=len(saved_SIDs2),
                                    msg="HTML")
            elif is_response_pdf(response): # if pdf response
                save_pdf(pdf_response=response, sid=sid)
                saved_SIDs2.append(sid)
                print_loop_no(total_sids=TOTAL_SIDs, 
                                    sid=sid,itr=i+1,
                                    type_="Success",
                                    type_itr=len(saved_SIDs2),
                                    msg="PDF")
            else: # otherwise
                failed_SIDs2.append(sid)
                print_loop_no(total_sids=TOTAL_SIDs, 
                                    sid=sid,itr=i+1,
                                    type_="Failed",
                                    type_itr=len(failed_SIDs2))
        except:
            failed_SIDs2.append(sid)
            print_loop_no(total_sids=TOTAL_SIDs, 
                                sid=sid,itr=i+1,
                                type_="Failed",
                                type_itr=len(failed_SIDs2))
    else:
        print_loop_no(total_sids=TOTAL_SIDs, 
                                sid=sid,itr=i+1,
                                type_="Processed",
                                type_itr=len(saved_SIDs2) + len(failed_SIDs2))

In [189]:
print(f"Total leftover SIDs 2: {len(saved_SIDs2) + len(failed_SIDs2)}")
print(f"Total saved SIDs 2   : {len(saved_SIDs2)}")
print(f"Total failed SIDs 2  : {len(failed_SIDs2)}")

Total leftover SIDs 2: 6043
Total saved SIDs 2   : 4693
Total failed SIDs 2  : 1350
